In [8]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sktime.datasets import load_UCR_UEA_dataset
import pandas as pd

from utils import medoid_per_class

In [9]:

# Paso 1: Cargar FordA
X, y = load_UCR_UEA_dataset(name="FordA", return_X_y=True)
X = X.iloc[:, 0].apply(pd.Series)
y = np.where(y == -1, 0, 1)  # Asegura etiquetas 0/1
X.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.797172,-0.664392,-0.373015,0.040815,0.526936,0.984288,1.353120,1.578108,1.659251,1.640809,...,1.120756,0.722417,0.362068,0.092083,-0.081268,-0.212573,-0.391456,-0.664392,-1.073796,-1.564343
1,0.804855,0.634629,0.373474,0.038343,-0.340988,-0.740860,-1.109667,-1.395357,-1.570192,-1.619951,...,0.386403,0.049213,-0.258138,-0.510583,-0.683647,-0.773817,-0.785255,-0.714885,-0.560443,-0.319086
2,0.727985,0.111284,-0.499124,-1.068629,-1.578351,-1.990534,-2.302031,-2.503403,-2.585211,-2.550600,...,0.394463,0.463685,0.507735,0.517174,0.504588,0.476270,0.438513,0.394463,0.339400,0.255391
3,-0.234439,-0.502157,-0.732488,-0.946128,-1.139739,-1.323336,-1.490243,-1.607077,-1.620430,-1.506933,...,-0.952804,-0.929437,-0.922761,-0.929437,-0.909409,-0.835970,-0.695768,-0.478790,-0.188707,0.119736
4,-0.171328,-0.062285,0.235829,0.710396,1.239969,1.649823,1.876321,1.865535,1.703751,1.466467,...,0.776188,0.725496,0.697453,0.731967,0.808545,0.839823,0.733046,0.437520,-0.026585,-0.602213


In [20]:
# Paso 2: Separar datos
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
medoids = medoid_per_class(X_train, y_train)
X_train_array = X_train.to_numpy().astype(np.float32).reshape(-1, 1, X_train.shape[1])

array([[[ 2.7199309e+00,  2.9467738e+00,  2.8241141e+00, ...,
          4.4338188e-01,  5.8141124e-01,  6.2248290e-01]],

       [[-5.7558346e-01, -6.4086282e-01, -5.6965804e-01, ...,
          5.7016188e-01,  2.5462359e-01, -1.4105225e-01]],

       [[ 2.2331664e-02, -1.0108495e-02,  1.7300829e-03, ...,
          2.9868621e-01,  2.0334819e-01,  9.2126645e-02]],

       ...,

       [[ 2.3826183e-01,  1.7041159e-01,  9.8589592e-02, ...,
          2.4781749e-02, -1.6883972e-01, -2.7673820e-01]],

       [[-1.4020537e+00, -7.7428800e-01, -9.3372762e-02, ...,
          4.5500639e-01,  6.3549049e-02, -4.5716318e-01]],

       [[-4.2900425e-01, -9.3333817e-01, -1.3708128e+00, ...,
          5.2270937e-01,  7.4887550e-01,  9.3211961e-01]]], dtype=float32)

In [21]:

# Paso 3: Obtener parámetros de MiniRocket
from minirocket_multivariate_variable import fit
num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, biases = fit(X_train_array)  # (num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, biases)

In [22]:

# Paso 4: Transformar con trazas
phi_X_train, _ = transform_prime(X_train, X_train, parameters)
phi_X_test, _ = transform_prime(X_test, X_test, parameters)

# Paso 5: Entrenar clasificador
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(phi_X_train, y_train)

# Paso 6: Aplicar explicabilidad
from explicabilidad import minirocket_explain
X_to_explain = X_test[0:1]  # Una muestra para explicar
y_pred = clf.predict(phi_X_test[0:1])

# Background de SHAP
background_X = X_train[:100]

# Aplicar MiniRocket Explain
contribuciones_beta_j, pred = minirocket_explain(
    X_to_explain, clf, background_X,
    base_explanation_method='strato',
    baseline_policy='baricenter_opposite_class',
    parameters=parameters
)

# Paso 7: Comparar con StratoSHAP puro (punto a punto)
from explicabilidad import shap_explain_on_timeseries
contribuciones_puntuales = shap_explain_on_timeseries(
    clf, X_to_explain, background_X
)

# Paso 8: Evaluar similitud
# Alinear los vectores para r2
beta_strato_phi = np.array([contribuciones_beta_j[j] for j in sorted(contribuciones_beta_j.keys())])
beta_strato_point = np.array([contribuciones_puntuales[j] for j in sorted(contribuciones_puntuales.keys())])
r2 = r2_score(beta_strato_point, beta_strato_phi)
print(f"R² entre explicaciones: {r2:.4f}")



ModuleNotFoundError: No module named 'explicabilidad'